# Foreign Exchange Return Forecasting of Neighboring Countries based on Powerful Anchor Countries
# *SIADS 696: Milestone II*

### *By Team #2*

## Overview
- Defining Custom Functions
- Importing Data
- Applying Unsupervised Learning
- Applying Supervised Learning

## Setup

In [1]:
# Importing Packages
import country_converter as coco
import matplotlib.pyplot as plt
import xgboost as xgb
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
import csv

# Importing partial packages
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from xgboost import XGBClassifier

In [8]:
# Ignoring unnecessary warnings
warnings.filterwarnings("ignore")

## Defining Custom Functions

In [3]:
# Preprocessing column names to standardize
def standardize_column_names(dataframe):
    dataframe.columns = [
        str(column).lower().replace(" ", "_").replace(",", "")
        for column in dataframe.columns
    ]

    return dataframe

In [4]:
def clean(country):
    if pd.isna(country):
        return country
    country = str(country).strip()
    country = p.sub("", country)
    return re.sub(r",\s*The$", "", country, flags=re.I)

In [5]:
def short(country):
    if pd.isna(country):
        return pd.NA
    res = cc.convert(names=clean(country), to="name_short")
    if isinstance(res, (list, tuple)):
        res = res[0] if res else pd.NA
    if not res or res == "not found" or pd.isna(res):
        warnings.warn(f"Problem: {country}")
        return pd.NA
    return res

In [6]:
def xxxusd(code):
    x = f"{code}USD=X", f"USD{code}=X"
    return x

## Importing Data

In [7]:
# Importing Real GDP Purchasing Power Parity
# Note the Ranking; 1 = big country per region
country_gdp = pd.read_csv("../data/input/real_gdp_purchasing_power_parity_0.csv")

# Displaying the first 5 rows of the dataframe
country_gdp.head()

,name,slug,value,date_of_information,ranking,region
0,China,china,"$33,598,000,000,000",2024,1,East and Southeast Asia
1,United States,united-states,"$25,676,000,000,000",2024,2,North America
2,India,india,"$14,244,000,000,000",2024,3,South Asia
3,Russia,russia,"$6,089,000,000,000",2024,4,Central Asia
4,Japan,japan,"$5,715,000,000,000",2024,5,East and Southeast Asia


In [8]:
# Importing Fred Anchors
df_fred_world_gdp = pd.read_csv("../data/input/fred_anchors_0.csv")

# Preprocessing Fred Anchors
df_fred_world_gdp["observation_date"] = pd.to_datetime(
    df_fred_world_gdp["observation_date"]
)
df_fred_world_gdp["time_period"] = df_fred_world_gdp["observation_date"].dt.year
df_fred_world_gdp = df_fred_world_gdp.drop(columns=["observation_date"]).rename(
    columns={"NYGDPMKTPCDWLD": "world_gdp"}
)

# Displaying the first 5 rows of the dataframe
df_fred_world_gdp.head()

,world_gdp,time_period
0,1.366519e+12,1960
1,1.444156e+12,1961
2,1.545552e+12,1962
3,1.671283e+12,1963
4,1.832063e+12,1964


In [9]:
# Importing GDP
df_imf_gdp = pd.read_csv(
    "../data/input/imf_gdp_0.csv",
    on_bad_lines="skip",
    sep=",",
    quoting=csv.QUOTE_ALL,
    skipinitialspace=True,
    usecols=[
        "COUNTRY",
        "TIME_PERIOD",
        "TYPE_OF_TRANSFORMATION",
        "FREQUENCY",
        "OBS_VALUE",
        "INDICATOR",
    ],
    engine="python",
)

# Preprocessing GDP
df_imf_gdp = (
    standardize_column_names(df_imf_gdp)
    .query(
        "indicator == 'US Dollar per domestic currency' "
        "and type_of_transformation == 'End-of-period (EoP)' "
        "and frequency == 'Annual'"
    )
    .dropna(subset=["obs_value"])
    .rename(columns={"time_period": "year"})
    .assign(year=lambda d: d["year"].str[:4])
    .sort_values(["country", "year"])
    .drop(columns=["indicator", "type_of_transformation", "frequency"])
    .reset_index(drop=True)
).assign(year=lambda d: pd.to_datetime(d["year"], errors="coerce").dt.year)

# Displaying the first 5 rows of the dataframe
df_imf_gdp.head()

,country,year,obs_value
0,"Afghanistan, Islamic Republic of",1955,0.059081
1,"Afghanistan, Islamic Republic of",1956,0.059081
2,"Afghanistan, Islamic Republic of",1957,0.059081
3,"Afghanistan, Islamic Republic of",1958,0.059081
4,"Afghanistan, Islamic Republic of",1959,0.059081


In [10]:
# Importing Exchange Rates
df_imf_trade = pd.read_csv(
    "../data/input/imf_trade_0.csv",
    usecols=[
        "COUNTRY",
        "COUNTERPART_COUNTRY",
        "TIME_PERIOD",
        "OBS_VALUE",
        "TRADE_FLOW",
        "SCALE",
        "UNIT",
    ],
)

# Manually excluding countries that are either trade in pegged currencies or don't have a true boundary from another
ls_0 = [
    "World",
    "Advanced Economies",
    "Latin America and the Caribbean (LAC)",
    "Hong Kong Special Administrative Region, People's Republic of China",
    "Emerging and Developing Europe",
    "Middle East and Central Asia",
    "Emerging Market and Developing Economies",
    "Euro Area (EA)",
    "Emerging and Developing Asia",
]

# Excluding the "Exclusion List"
df_imf_trade = df_imf_trade[
    (~df_imf_trade["COUNTRY"].isin(ls_0))
    & (~df_imf_trade["COUNTERPART_COUNTRY"].isin(ls_0))
]

# Viewing first five rows
df_imf_trade = standardize_column_names(df_imf_trade).sort_values(
    by=["obs_value"], ascending=False
)

# Displaying the first 5 rows of the dataframe
df_imf_trade.head()

,country,counterpart_country,time_period,obs_value,scale,trade_flow,unit
400545,"China, People's Republic of",United States,2021-Q4,1.651331e+11,Millions,Exports of goods,US dollar
400544,"China, People's Republic of",United States,2021-Q3,1.590074e+11,Millions,Exports of goods,US dollar
400548,"China, People's Republic of",United States,2022-Q3,1.558462e+11,Millions,Exports of goods,US dollar
400547,"China, People's Republic of",United States,2022-Q2,1.542070e+11,Millions,Exports of goods,US dollar
400557,"China, People's Republic of",United States,2024-Q4,1.430223e+11,Millions,Exports of goods,US dollar


In [11]:
# Defining countries in the European Union that use the Euro (EUR) and British Pound (GBP)
EUR = [
    "Austria",
    "Belgium",
    "Cyprus",
    "Estonia",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Ireland",
    "Italy",
    "Latvia",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Netherlands",
    "Portugal",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Andorra",
    "Monaco",
    "San Marino",
    "Vatican City",
    "Saint Barthélemy",
    "Saint Pierre and Miquelon",
    "Kosovo",
    "Montenegro",
    "Bosnia and Herzegovina",
    "Bulgaria",
    "Cape Verde",
    "Cameroon",
    "Central African Republic",
    "Chad",
    "Republic of the Congo",
    "Equatorial Guinea",
    "Gabon",
    "Benin",
    "Burkina Faso",
    "Côte d'Ivoire",
    "Guinea-Bissau",
    "Mali",
    "Niger",
    "Senegal",
    "Togo",
    "French Polynesia",
    "New Caledonia",
    "Wallis and Futuna",
    "Comoros",
    "Croatia",
    "Morocco",
    "São Tomé and Príncipe",
    "Denmark",
    "North Macedonia",
]

# Defining countries that use the British Pound (GBP)
GBP = [
    "Guernsey",
    "Jersey",
    "Isle of Man",
    "Gibraltar",
    "Falkland Islands",
    "Saint Helena",
]

# Defining other countries with pegged currencies or no true boundary
other = [
    "Bhutan",
    "Nepal",
    "North Korea",
    "Afghanistan",
    "Turkmenistan",
    "South Sudan",
    "Guam",
    "Macau",
    "Tuvalu",
    "Kiribati",
    "Palau",
    "Greenland",
    "Maldives",
    "Iraq",
    "Solomon Islands",
    "Brunei Darussalam",
    "Bangladesh",
    "Myanmar",
    "Marshall Islands",
    "Iran",
    "Yemen",
    "Libya",
    "Somalia",
    "Liberia",
    "Sudan",
    "Sierra Leone",
    "Mongolia",
    "Angola",
    "Kyrgyz Republic",
    "Tajikistan",
]

# Cleaning country names
EUR = {short(x) for x in EUR if pd.notna(short(x))}
GBP = {short(x) for x in GBP if pd.notna(short(x))}
other = {short(x) for x in other if pd.notna(short(x))}

NameError: name 'cc' is not defined

In [ ]:
# Importing cleaned data
# TODO - where is this coming from? I.e. what code created these files?
df_trade = pd.read_parquet("../data/input/trade_0.parquet")
df_gdp = pd.read_parquet("../data/input/gdp_0.parquet")

## Applying Unsupervised Learning

In [ ]:
# Converting country names to short format
cc = coco.CountryConverter()
p = re.compile(
    r",\s*(?:Kingdom of the Netherlands|United Kingdom-British Overseas Territory|Republic of the|Union of the|State of the)$",
    re.I,
)

In [ ]:
# Anchors selection
df_gdp = df_gdp[~(df_gdp["COUNTRY"] == "United States")]
df_gdp.head()
Anchor = {
    "EUR": EUR,
    "CNY": {"China"},
    "JPY": {"Japan"},
}

# No USD due to many relationship ties with multiple currency.
# Time Period
period = 10

# N Neighbor
n = 10

# Minimum Percentage Volume
v = 0.05

# Exports of goods
# Neighbor mapping
neighbor = {x: i for i, y in Anchor.items() for x in y}
ex = EUR | GBP | other | {"China", "Japan", "United States"}
df_trade = df_trade.dropna(
    subset=["COUNTRY", "COUNTERPART_COUNTRY", "TIME_PERIOD", "OBS_VALUE"]
).copy()

# Keep data period
df_trade = df_trade[df_trade["TIME_PERIOD"].astype(str).str.match(r"^\d{4}")]
df_trade["year"] = df_trade["TIME_PERIOD"].astype(str).str[:4].astype(int)
max_year = int(df_trade["year"].max())
df_trade = df_trade[df_trade["year"].between(max_year - period + 1, max_year)]

# Filter and map potential neighbors
df_export = df_trade[~df_trade["COUNTRY"].isin(ex)].copy()
df_export["anchor"] = df_export["COUNTERPART_COUNTRY"].map(neighbor)

# Total exports per country
etot = (
    df_export.groupby("COUNTRY", as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "export_total"})
)

# Exports to each anchor
eanch = (
    df_export.dropna(subset=["anchor"])
    .groupby(["COUNTRY", "anchor"], as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "export_to_anchor"})
)

# Export shares
esh = eanch.merge(etot, on="COUNTRY", how="left").assign(
    export_share=lambda d: d["export_to_anchor"] / d["export_total"]
)

# Best export anchor per country
bexp = esh.sort_values(
    ["COUNTRY", "export_share", "export_to_anchor"], ascending=[True, False, False]
).drop_duplicates("COUNTRY")
bexp = bexp[bexp["export_share"] >= v]

# Swap role
df_import = df_trade.rename(
    columns={"COUNTRY": "COUNTERPART_COUNTRY_orig", "COUNTERPART_COUNTRY": "COUNTRY"}
).rename(columns={"COUNTERPART_COUNTRY_orig": "COUNTERPART_COUNTRY"})

# Potential neighbors
df_import = df_import[~df_import["COUNTRY"].isin(ex)].copy()
df_import["anchor"] = df_import["COUNTERPART_COUNTRY"].map(neighbor)

# Total imports per country
import_totals = (
    df_import.groupby("COUNTRY", as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "import_total"})
)

# Imports from each anchor
ianch = (
    df_import.dropna(subset=["anchor"])
    .groupby(["COUNTRY", "anchor"], as_index=False)["OBS_VALUE"]
    .sum()
    .rename(columns={"OBS_VALUE": "import_from_anchor"})
)

# Import shares
ish = ianch.merge(import_totals, on="COUNTRY", how="left").assign(
    import_share=lambda d: d["import_from_anchor"] / d["import_total"]
)

# Best import anchor per country
bimp = ish.sort_values(
    ["COUNTRY", "import_share", "import_from_anchor"], ascending=[True, False, False]
).drop_duplicates("COUNTRY")
bimp = bimp[bimp["import_share"] >= v]

In [ ]:
# Combine table
comb = esh.merge(ish, on=["COUNTRY", "anchor"], how="outer", suffixes=("_exp", "_imp"))

# Handle missing value
f = [
    "export_to_anchor",
    "export_total",
    "export_share",
    "import_from_anchor",
    "import_total",
    "import_share",
]
for c in f:
    if c not in comb.columns:
        comb[c] = 0.0
comb[f] = comb[f].fillna(0.0)

# Combined metrics
comb = comb.assign(
    total_trade_with_anchor=lambda d: d["export_to_anchor"] + d["import_from_anchor"],
    total_trade_volume=lambda d: d["export_total"] + d["import_total"],
)
comb["combined_exposure"] = 0.0
nz = comb["total_trade_volume"] > 0
comb.loc[nz, "combined_exposure"] = (
    comb.loc[nz, "total_trade_with_anchor"] / comb.loc[nz, "total_trade_volume"]
)
comb_f = comb.loc[comb["combined_exposure"] >= v].copy()

# Rank neighbors per anchor
comb_f = comb_f.sort_values(
    by=[
        "anchor",
        "combined_exposure",
        "total_trade_with_anchor",
        "export_share",
        "import_share",
        "COUNTRY",
    ],
    ascending=[True, False, False, False, False, True],
)

# Rank within each anchor and take top n
comb_f["rank_within_anchor"] = comb_f.groupby("anchor")["combined_exposure"].rank(
    method="first", ascending=False
)
comb_top = comb_f.loc[comb_f["rank_within_anchor"] <= n].copy()

# Final neighbor
neighbors_dict = comb_top.groupby("anchor")["COUNTRY"].apply(list).to_dict()

# Summary results
summary = (
    comb_top[
        [
            "anchor",
            "COUNTRY",
            "combined_exposure",
            "export_share",
            "import_share",
            "export_to_anchor",
            "import_from_anchor",
            "export_total",
            "import_total",
            "total_trade_with_anchor",
            "total_trade_volume",
            "rank_within_anchor",
        ]
    ]
    .sort_values(["anchor", "rank_within_anchor"])
    .reset_index(drop=True)
)

res = summary.copy()
for col in ["combined_exposure", "export_share", "import_share"]:
    res[col] = (res[col] * 100).round(2)

res_sorted = res.sort_values(["anchor", "rank_within_anchor"])
anchor_neighbor = res_sorted.groupby("anchor")["COUNTRY"].apply(list).to_dict()

for a, c in [("CNY", "Australia"), ("JPY", "Philippines"), ("JPY", "Vietnam")]:
    if a in anchor_neighbor and c in anchor_neighbor[a]:
        anchor_neighbor[a].remove(c)

anchor_neighbor

*Performing Principle Component Analyis (PCA) to reduce the dimensionality of our training data.*

In [ ]:
# We identify the hour anchor: Anchor = {"EUR": EUR, "CNY": {"China"}, "JPY": {"Japan"}}

# Going to start by pulling Anchor Countries
# Codes are obtain from yahoo finance symbol
# https://finance.yahoo.com/markets/currencies/

fx_list = {
    # Anchor currencies
    "CNY": "CNY=X",  # China
    "EUR": "EUR=X",  # Euro
    "JPY": "JPY=X",  # Japan
    # Neutral currencies
    "CAD": "CAD=X",  # Canada
    "BRL": "BRL=X",  # Brazil
    "MXN": "MXN=X",  # Mexico
    "COP": "COP=X",  # Colombia
    "PEN": "PEN=X",  # Peru
    "NOK": "NOK=X",  # Norway
    "ZAR": "ZAR=X",  # South Africa
    "INR": "INR=X",  # India
    "TRY": "TRY=X",  # Turkey
    "EGP": "EGP=X",  # Egypt
    "RUB": "RUB=X",  # Russia
    "ILS": "ILS=X",  # Israel
    # CNY group
    "CDF": "CDF=X",  # DR Congo
    "LAK": "LAK=X",  # Laos
    "TZS": "TZS=X",  # Tanzania
    "CLP": "CLP=X",  # Chile
    "GNF": "GNF=X",  # Guinea
    "PKR": "PKR=X",  # Pakistan
    "PHP": "PHP=X",  # Philippines
    "VND": "VND=X",  # Vietnam
    "MRU": "MRU=X",  # Mauritania
    # EUR group
    "ALL": "ALL=X",  # Albania
    "CZK": "CZK=X",  # Czechia
    "TND": "TND=X",  # Tunisia
    "RON": "RON=X",  # Romania
    "HUF": "HUF=X",  # Hungary
    "PLN": "PLN=X",  # Poland
    "RSD": "RSD=X",  # Serbia
    "SEK": "SEK=X",  # Sweden
    "ISK": "ISK=X",  # Iceland
    "DZD": "DZD=X",  # Algeria
    # JPY group
    "PGK": "PGK=X",  # Papua New Guinea
    "TWD": "TWD=X",  # Taiwan
    "THB": "THB=X",  # Thailand
    "AUD": "AUD=X",  # Australia
    "IDR": "IDR=X",  # Indonesia
    "KRW": "KRW=X",  # South Korea
    "MYR": "MYR=X",  # Malaysia
    "NZD": "NZD=X",  # New Zealand
}

# Period is the timeframe we want, we conclude for the time being, it will be 10 years
PERIOD = "10y"

# We conclude we will be looking at exchanges on a daily basis.
INTERVAL = "1d"
FFill_Limit = 3
N_PCS_To_Cluster = 3
N_Cluster = 3

# Creating a list of tickers from the fx_list dictionary
tickers = list(fx_list.values())

# Creating a DataFrame by downloading historical FX data using yfinance
yfinance_fx_raw = yf.download(
    tickers, period=PERIOD, interval=INTERVAL, auto_adjust=None, progress=False
)

In [ ]:
# Remove problem currency
x = yfinance_fx_raw["Close"]
c = x.notna().sum().sort_values()
c

In [ ]:
# We remove currencies that have less than 2000 data points
want = list(fx_list.keys())
t = []
invert = {}

# Loop through each currency in the want list and get their corresponding ticker symbols
for c in want:
    t1, t2 = xxxusd(c)
    t.extend([t1, t2])

# Remove duplicates while preserving order
t = list(dict.fromkeys(t))

# Creating a DataFrame by downloading historical FX data using yfinance
raw = yf.download(
    t, period=PERIOD, interval=INTERVAL, auto_adjust=None, progress=False
)["Close"]

# Creating a DataFrame to hold exchange rate levels and whether they need to be inverted
level = {}
for c in want:
    t1, t2 = xxxusd(c)
    if t1 in raw.columns and raw[t1].notna().sum() > 0:
        level[c] = raw[t1]
        invert[c] = False
    elif t2 in raw.columns and raw[t2].notna().sum() > 0:
        level[c] = 1.0 / raw[t2]
        invert[c] = True

# Creating a DataFrame from the level dictionary and sorting by index
level = pd.DataFrame(level).sort_index()

# Calculating log returns and handling missing values
lreturn = np.log(level / level.shift(1))
lreturn = lreturn.ffill(limit=3).dropna(how="any")

In [ ]:
# Standardizing the log returns
scaler = StandardScaler()
X = pd.DataFrame(
    scaler.fit_transform(lreturn),
    index=lreturn.index,
    columns=lreturn.columns,
)

# PCA and KMeans Clustering
pca = PCA(n_components=2)
Z = pca.fit_transform(X.T)

# Creating a DataFrame for PCA results
pca_df = pd.DataFrame(Z, index=X.columns, columns=["PC1", "PC2"])

# KMeans Clustering
k = 3
km = KMeans(n_clusters=k, random_state=42)
pca_df["cluster"] = km.fit_predict(pca_df[["PC1", "PC2"]].to_numpy())

In [ ]:
# Addidtional pull for explained_variance_ratio_
evr = pca.explained_variance_ratio_
print("Explained variance ratio: ", evr, " | Cumalative: ", evr.cumsum())

In [ ]:
# Grouping currencies based on economic ties
g = {
    "CNY": ["CDF", "LAK", "TZS", "CLP", "GNF", "PKR", "PHP", "VND", "MRU"],
    "EUR": ["ALL", "CZK", "TND", "RON", "HUF", "PLN", "RSD", "SEK", "ISK", "DZD"],
    "JPY": ["PGK", "TWD", "THB", "AUD", "IDR", "KRW", "MYR", "NZD"],
}


# Function to assign group based on currency code
def assign(cc):
    for i, j in g.items():
        if cc == i or cc in j:
            return i
    return "other"


# Assigning group to each currency in the PCA DataFrame
pca_df["group"] = pca_df.index.map(assign)

# Visualization of PCA Results
col = {"CNY": "red", "EUR": "blue", "INR": "green", "JPY": "orange", "other": "gray"}
plt.figure(figsize=(12, 8))
for grp, sub in pca_df.groupby("group"):
    plt.scatter(
        sub["PC1"],
        sub["PC2"],
        c=col.get(grp, "black"),
        s=70,
        alpha=0.85,
        label=f"{grp} (n={len(sub)})",
    )
    for i, j in sub.iterrows():
        plt.annotate(
            i,
            (j["PC1"], j["PC2"]),
            xytext=(3, 3),
            textcoords="offset points",
            fontsize=8,
        )

plt.title("Currencies in PCA Space (standardized returns, XXX/USD)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
# Calculating Silhouette Score
score = silhouette_score(pca_df[["PC1", "PC2"]], pca_df["cluster"])

# Saving the log returns DataFrame to a Parquet file
lreturn.to_parquet("fx_log_return.parquet", index=False)

## Applying Supervised Learning

*Applying Granger causality test to determine if a causal realtionship is found in our anchor and neighboring currency returns*

In [2]:
lreturn = pd.read_parquet("../data/input/fx_log_return.parquet")

g = {
    "CNY": ["CDF", "LAK", "TZS", "CLP", "GNF", "PKR", "PHP", "VND", "MRU"],
    "EUR": ["ALL", "CZK", "TND", "RON", "HUF", "PLN", "RSD", "SEK", "ISK", "DZD"],
    "JPY": ["PGK", "TWD", "THB", "AUD", "IDR", "KRW", "MYR", "NZD"],
}


def granger(res, g, ml=5, a=0.05):
    x = []
    for i, j in g.items():
        for k in j:
            if i not in res.columns or k not in res.columns:
                continue
            df = pd.concat([res[k], res[i]], axis=1).dropna()
            df.columns = ["neighbor", "anchor"]
            if len(df) <= ml * 3:
                continue
            t = grangercausalitytests(df, maxlag=ml)
            pval = [t[lag][0]["ssr_ftest"][1] for lag in range(1, ml + 1)]
            min_p = float(np.min(pval))
            best_lag = int(np.argmin(pval) + 1)
            x.append(
                {
                    "anchor": i,
                    "neighbor": k,
                    "min_pval": round(min_p, 4),
                    "best_lag (days)": best_lag,
                    "significant": (min_p < a),
                }
            )
    rdf = pd.DataFrame(x)
    return rdf


rdf = granger(lreturn, g, 10, 0.05)
rdf = rdf.sort_values(["anchor", "min_pval"]).reset_index(drop=True)
an = rdf[rdf["significant"] == True].sort_values(["anchor", "min_pval"])
ng = an.groupby("anchor")["neighbor"].apply(list).to_dict()
ng


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=6.2637  , p=0.0124  , df_denom=1799, df_num=1
ssr based chi2 test:   chi2=6.2741  , p=0.0123  , df=1
likelihood ratio test: chi2=6.2632  , p=0.0123  , df=1
parameter F test:         F=6.2637  , p=0.0124  , df_denom=1799, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.1649  , p=0.0425  , df_denom=1796, df_num=2
ssr based chi2 test:   chi2=6.3474  , p=0.0418  , df=2
likelihood ratio test: chi2=6.3362  , p=0.0421  , df=2
parameter F test:         F=3.1649  , p=0.0425  , df_denom=1796, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.8108  , p=0.1431  , df_denom=1793, df_num=3
ssr based chi2 test:   chi2=5.4537  , p=0.1414  , df=3
likelihood ratio test: chi2=5.4455  , p=0.1419  , df=3
parameter F test:         F=1.8108  , p=0.1431  , df_denom=1793, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.4194  , p=0.

{'CNY': ['VND', 'MRU', 'CDF', 'PKR'],
 'EUR': ['ALL', 'CZK', 'TND', 'RON', 'RSD', 'DZD', 'HUF', 'SEK', 'ISK', 'PLN'],
 'JPY': ['MYR', 'TWD', 'THB', 'IDR', 'AUD']}

In [17]:
ls_0 = [i for i in ng.items() if i[0] == "EUR"][0]
ls_1 = [ls_0[0] for i in range(len([i for i in ng.items() if i[0] == "EUR"][0][1]))]
len(ls_0[1]), len(ls_1)

xgb_df = pd.DataFrame({"anchor": ls_1, "neighbor": ls_0[1]})

In [ ]:
def create_lag(neighbor, dataframe):
    # Create lag features (e.g., previous 5 days' returns)
    for i in range(1, 25):
        dataframe[f"return_lag_{i}"] = dataframe[neighbor].pct_change().shift(i)

    # Create moving average features
    dataframe["MA_30"] = dataframe[neighbor].rolling(window=30).mean().shift(1)
    dataframe["MA_60"] = dataframe[neighbor].rolling(window=60).mean().shift(1)
    dataframe["MA_90"] = dataframe[neighbor].rolling(window=90).mean().shift(1)

    # Drop initial rows with NaNs created by rolling windows
    dataframe.dropna(inplace=True)
    dataframe = dataframe.replace([np.inf, -np.inf], np.nan).fillna(0)

    return dataframe

# Run this for the neighbor alone: control
# Run this for the anchor + neighbor: treatment
def perform_xgb_for_anchor_neighbor(neighbor, dataframe):
    # Starting with CNY/CDF as an example
    dataframe["target"] = dataframe[neighbor].shift(-1)

    # Define features (X) and target (y)
    features = [
        col
        for col in df_0.columns
        if col == neighbor or col.startswith("MA_") or col.startswith("return_lag_")
    ]
    X = dataframe[features]
    y = dataframe["target"]

    # Time-based train-test split
    split_index = int(len(X) * 0.8)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    xgb_0 = xgb.XGBRegressor()
    parameters = {
        "nthread": [4],
        "objective": ["reg:squarederror"],
        "learning_rate": [0.08, 0.15, 0.3],
        "max_depth": [2, 4, 8],
        "min_child_weight": [4],
        "subsample": [0.2, 0.5, 0.8],
        "colsample_bytree": [0.7],
        "n_estimators": [2, 5, 10],
    }

    xgb_grid = GridSearchCV(xgb_0, parameters, cv=2, n_jobs=5, verbose=True)
    xgb_grid.fit(X_train, y_train)

    return xgb_grid.best_score_, xgb_grid.best_params_

In [ ]:
# def visualize_xgb_tree(xgb_grid):

#     # Visualizing the XGBoost model
#     fig, ax = plt.subplots(figsize=(2, 5))
#     xgb.plot_tree(xgb_grid, ax=ax)

#     return plt.show()

In [ ]:
df_0 = pd.read_parquet("../data/input/fx_log_return.parquet")

for i, j in zip(xgb_df["anchor"], xgb_df["neighbor"]):
    df_1 = df_0[[i, j]]
    df_1.dropna(inplace=True)
    df_1 = create_lag(j, df_1).drop(columns=[i])
    score, params = perform_xgb_for_anchor_neighbor(j, df_1)

    print(f"Anchor: {i}, Neighbor: {j}, Score: {score}, Params: {params}")

Fitting 2 folds for each of 81 candidates, totalling 162 fits


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 189, in _run_module_as_main
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "/Users/josephhiggins/Documents/siads-696/venv/lib/python3.12/site-packages/joblib/__init__.py", line 115, in <module>
  File "<frozen runpy>", line 112, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "/Users/josephhiggins/Documents/siads-696/venv/lib/python3.12/site-packages/joblib/__init__.py", line 115, in <module>
  File "<frozen runpy>", line 112, in _get_module_details
  File "/Users/josephhiggins/Documents/siads-696/ve

KeyboardInterrupt: 